In [8]:
#!pip install sentence-transformers

In [9]:
# Import necessary libraries
import pandas as pd
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.pipeline import make_pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import SVC
from sklearn.metrics import classification_report
from sentence_transformers import SentenceTransformer
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV
import joblib
import re
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer

In [10]:
# Download NLTK stopwords
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [11]:
# Step 1: Load and preprocess data
df1 = pd.read_csv("/content/articles.csv", encoding='latin1')  # Update the file path and encoding
df = df1[['Full_Article', 'Article_Type']]  # Select relevant columns

In [65]:
df1

,Id,Heading,Article.Banner.Image,Outlets,Article.Description,Full_Article,Article_Type,Tonality
0,d6995462-5e87-453b-b64d-e9f1df6e94d2,"A Puzzling Maneuver, Then Freefall: NTSB Report Provides New Details in Southeast Alaska Helicopter Crash That Killed 3",NaN,Essex Caller,"<p>The helicopter that crashed in Southeast Alaska in late September, killing three people, entered a 500-foot freefall before dropping to a Glacier Bay National Park beach, according to by the National Transportation Safety Board. The preliminary NTSB report released Friday offers no official probable cause. That determination won&lsquo;t be made until next year at the earliest.</p>","<p>The helicopter that crashed in Southeast Alaska in late September, killing three people, entered a 500-foot freefall before dropping to a Glacier Bay National Park beach, according to by the National Transportation Safety Board.&nbsp;The preliminary NTSB report released Friday offers no official probable cause. That determination won&lsquo;t be made until next year at the earliest.</p>",Commercial,Negative
1,8b05e939-a89e-4548-b92b-013822e8ee7d,Bells Nexus Air Taxi Concept Rings Changes For eVTOL Market,NaN,Aviation Week Network,"<p>A year after teasing the fledgling electric vertical-takeoff-and-landing (eVTOL) industry with a mockup of a futuristic air taxi cabin, rotary-wing giant Bell has unveiled the first major configuration details of its Nexus on-demand urban air mobility (UAM) concept. Revealed at the Consumer Technology Association&rsquo;s CES 2019 show in Las Vegas, Bell&rsquo;s Nexus is distinguished by six tilting ducted fans and sized to carry four passengers and a pilot.&nbsp;</p>","<p>A year after teasing the fledgling electric vertical-takeoff-and-landing (eVTOL) industry with a mockup of a futuristic air taxi cabin, rotary-wing giant Bell has unveiled the first major configuration details of its Nexus on-demand urban air mobility (UAM) concept. Revealed at the Consumer Technology Association&rsquo;s CES 2019 show in Las Vegas, Bell&rsquo;s Nexus is distinguished by six tilting ducted fans and sized to carry four passengers and a pilot.&nbsp;</p>",Commercial,Positive
2,69fcd400-bceb-4255-8277-619f2d68ac0b,Bell Helicopter Show Air Taxi Nexus,http://images.tmtpost.com/uploads/images/2019/01/20190108074154855.png,TMTPost,"<p>Bell released the full-size design of the vertical take-off and landing ( VTOL ) air taxi during CES 2019. The new hybrid air taxi was named Nexus. It connects transportation, technology and comfort and convenience.</p>","<p>Bell released the full-size design of the vertical take-off and landing ( VTOL ) air taxi during CES 2019. The new hybrid air taxi was named Nexus. It connects transportation, technology and comfort and convenience.</p>",Commercial,Positive
3,17943578-c11b-414b-b3f5-063d3a93157b,BELL DÉVOILE LA CONCEPTION INTÉGRALE DE SON TAXI AÉRIEN,http://www.fredzone.org/wp-content/uploads/2019/01/bell-2-640x336.jpg,Fredzone,"<p>Bell est une soci&eacute;t&eacute; am&eacute;ricaine dirig&eacute;e par Mitch Snyder. La start-up a abandonn&eacute; la construction d&rsquo;h&eacute;licopt&egrave;res pour se consacrer essentiellement aux nouvelles solutions de mobilit&eacute;s urbaines. Le 07 janvier 2019, le CEO d&eacute;voile au public &laquo; Bell Nexus. &raquo; Il &lsquo;agit d&rsquo;un taxi a&eacute;rien hybride &agrave; cinq places capable de d&eacute;coller et atterrir &agrave; la verticale.</p>","<p>Bell est une soci&eacute;t&eacute; am&eacute;ricaine dirig&eacute;e par Mitch Snyder. La start-up a abandonn&eacute; la construction d&rsquo;h&eacute;licopt&egrave;res pour se consacrer essentiellement aux nouvelles solutions de mobilit&eacute;s urbaines. Le 07 janvier 2019, le CEO d&eacute;voile au public &laquo; Bell Nexus. &raquo; Il &lsquo;agit d&rsquo;un taxi a&eacute;rien hybride &agrave; cinq places capable de d&eacute;coller et atterrir &agrave; la verticale.</p>",Commercial,Positive
4,f33c7b11-5f77-4a98-bb2e-d36689042aea,Les 

In [12]:
# Step 1.1: Clean and preprocess text data
def preprocess_text(text):
    # Remove HTML tags
    text = re.sub(r'<[^>]+>', '', text)
    # Remove special characters and digits using regex
    text = re.sub(r'[^a-zA-Z]', ' ', text)
    # Convert to lowercase
    text = text.lower()
    # Remove stopwords
    stop_words = set(stopwords.words('english'))
    text = ' '.join([word for word in text.split() if word not in stop_words])
    # Stemming
    stemmer = PorterStemmer()
    text = ' '.join([stemmer.stem(word) for word in text.split()])
    return text

In [13]:
df['Cleaned_Article'] = df['Full_Article'].apply(preprocess_text)

<ipython-input-13-f8d984fbd378>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Cleaned_Article'] = df['Full_Article'].apply(preprocess_text)


In [18]:
# Step 2: Tokenize and vectorize

# Download the model if it's not already locally stored
!wget https://huggingface.co/sentence-transformers/distilbert-base-multilingual-cased/resolve/distilbert-base-multilingual-cased.zip

# Extract the model files
!unzip distilbert-base-multilingual-cased.zip

# Initialize the SentenceTransformer model using the local model files
bert_model = SentenceTransformer('distilbert-base-multilingual-cased')

# Encode and vectorize the text data
X = bert_model.encode(df['Cleaned_Article'], convert_to_tensor=True)

--2023-11-28 19:37:20--  https://huggingface.co/sentence-transformers/distilbert-base-multilingual-cased/resolve/distilbert-base-multilingual-cased.zip
Resolving huggingface.co (huggingface.co)... 65.8.243.46, 65.8.243.92, 65.8.243.90, ...
Connecting to huggingface.co (huggingface.co)|65.8.243.46|:443... connected.
HTTP request sent, awaiting response... 404 Not Found
2023-11-28 19:37:20 ERROR 404: Not Found.

unzip:  cannot find or open distilbert-base-multilingual-cased.zip, distilbert-base-multilingual-cased.zip.zip or distilbert-base-multilingual-cased.zip.ZIP.


.gitattributes:   0%|          | 0.00/399 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/7.32k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/466 [00:00<?, ?B/s]

model.onnx:   0%|          | 0.00/909M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/542M [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/542M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/996k [00:00<?, ?B/s]

In [19]:
# Step 3: Encode the target variable (Article_Type) using LabelEncoder
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(df['Article_Type'])

In [20]:
# Step 4: Select and train an ML classifier model
model = make_pipeline(StandardScaler(), SVC())
model.fit(X, y)

Pipeline(steps=[('standardscaler', StandardScaler()), ('svc', SVC())])

In [21]:
# Step 5: Hyperparameter tuning using GridSearchCV
param_grid = {'svc__C': [0.1, 1, 10], 'svc__kernel': ['linear', 'rbf']}
grid_search = GridSearchCV(model, param_grid, cv=5)
grid_search.fit(X, y)
best_model = grid_search.best_estimator_

In [22]:
# Step 6: Validate and evaluate using cross-validation
cross_val_scores = cross_val_score(best_model, X, y, cv=5)
print("Cross-validation scores:", cross_val_scores)
print("Mean accuracy:", cross_val_scores.mean())

Cross-validation scores: [0.88385598 0.86875726 0.84320557 0.82926829 0.83739837]
Mean accuracy: 0.8524970963995354


In [23]:
# Step 7: Save and reload the model
joblib.dump(best_model, 'text_classification_model.joblib')  # Save model
loaded_model = joblib.load('text_classification_model.joblib')  # Reload model

In [60]:
import pandas as pd
import re

# Read the CSV file into a DataFrame
unknown_df = pd.read_csv('/content/unknown_articles.csv')

# Assume the column containing URLs is named 'Article.URL'
urls_column = 'Article.URL'

# Convert the URL column to string
unknown_df[urls_column] = unknown_df[urls_column].astype(str)
unknown_df['Article_Name'] = unknown_df[urls_column].apply(lambda url: re.search(r'/([^/]+)/?$', url).group(1) if re.search(r'/([^/]+)/?$', url) else None)


# Extract article names and remove ".html" extension
#unknown_df['Article_Name'] = unknown_df[urls_column].apply(lambda url: re.search(r'/([^/]+)\.html$', url).group(1) if re.search(r'/([^/]+)\.html$', url) else None)


In [61]:
unknown_df

,Article.URL,Article_Name
0,http://australianaviation.com.au/2018/10/a-competitive-edge-50-years-of-the-australian-army-aviation-corps/,a-competitive-edge-50-years-of-the-australian-army-aviation-corps
1,http://australianaviation.com.au/2018/10/victoria-police-orders-four-aircraft-for-air-wing-fleet/,victoria-police-orders-four-aircraft-for-air-wing-fleet
2,http://australianaviation.com.au/2018/10/army-aviation-celebrates-50-years-farewells-the-kiowa/,army-aviation-celebrates-50-years-farewells-the-kiowa
3,https://attain.news/community/special-sea-king-flypast-at-the-royal-junior-school,special-sea-king-flypast-at-the-royal-junior-school
4,https://m.ariva.de/amp/ad-hoc-airbus-board-of-directors-selects-guillaume-faury-7190521,ad-hoc-airbus-board-of-directors-selects-guillaume-faury-7190521
...,...,...
893,https://www.verticalmag.com/press-releases/airbus-to-showcase-h135-as-future-navy-helicopter-trainer/,airbus-to-showcase-h135-as-future-navy-helicopter-trainer
894,https://www.verticalmag.com/press-releases/metro-aviation-sparks-new-era-of-helicopter-safety-at-amtc/,metro-aviation-sparks-new-era-of-helicopter-safety-at-amtc
895,https://www.verticalmag.com/news/helicopter-ems-market-idling-in-china/,helicopter-ems-market-idling-in-china
896,https://www.verticalmag.com/news/airbus-optimistic-for-future-of-north-american-hems-market/,airbus-optimistic-for-future-of-north-american-hems-market


In [75]:
unknown_df['Cleaned_Article'] = unknown_df['Article_Name'].apply(preprocess_text)  # Assuming 'Article_Name' is the relevant column

# Encode and vectorize the text data for unknown data
X_unknown = bert_model.encode(unknown_df['Cleaned_Article'], convert_to_tensor=True)

Make predictions for unknown data
predicted_types_unknown = loaded_model.predict(X_unknown)

# Display the predicted types for unknown data
unknown_df['Predicted_Article_Type'] = predicted_types_unknown
print(unknown_df[['Article_Name', 'Predicted_Article_Type']])

                                                          Article_Name  \
0    a-competitive-edge-50-years-of-the-australian-army-aviation-corps   
1              victoria-police-orders-four-aircraft-for-air-wing-fleet   
2                army-aviation-celebrates-50-years-farewells-the-kiowa   
3                  special-sea-king-flypast-at-the-royal-junior-school   
4     ad-hoc-airbus-board-of-directors-selects-guillaume-faury-7190521   
..                                                                 ...   
893          airbus-to-showcase-h135-as-future-navy-helicopter-trainer   
894         metro-aviation-sparks-new-era-of-helicopter-safety-at-amtc   
895                              helicopter-ems-market-idling-in-china   
896         airbus-optimistic-for-future-of-north-american-hems-market   
897   airbus-helicopters-to-showcase-ems-capabilities-at-helitech-2018   

     Predicted_Article_Type  
0                         0  
1                         0  
2                    

In [77]:
unknown_df['Predicted_Article_Type'].value_counts()

0    637
3    255
1      6
Name: Predicted_Article_Type, dtype: int64

In [76]:
unknown_df[['Article_Name', 'Predicted_Article_Type']]

,Article_Name,Predicted_Article_Type
0,a-competitive-edge-50-years-of-the-australian-army-aviation-corps,0
1,victoria-police-orders-four-aircraft-for-air-wing-fleet,0
2,army-aviation-celebrates-50-years-farewells-the-kiowa,0
3,special-sea-king-flypast-at-the-royal-junior-school,3
4,ad-hoc-airbus-board-of-directors-selects-guillaume-faury-7190521,1
...,...,...
893,airbus-to-showcase-h135-as-future-navy-helicopter-trainer,0
894,metro-aviation-sparks-new-era-of-helicopter-safety-at-amtc,0
895,helicopter-ems-market-idling-in-china,0
896,airbus-optimistic-for-future-of-north-american-hems-market,0
